# GSE249279 Visium QC (Scanpy/Squidpy)

This notebook verifies that the Visium spatial sections extracted from `GSE249279_RAW.tar` load successfully, and produces basic QC plots.

In [ ]:
import json
from pathlib import Path

import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt

processed_dir = Path("../data/processed/spatial")
summary_path = processed_dir / "load_summary.json"

with open(summary_path) as f:
    summary = json.load(f)

len(summary), summary[0]


In [ ]:
# Load all sections (one AnnData per section)
ads = {}
for rec in summary:
    p = Path(rec["out_h5ad"])
    ad = sc.read_h5ad(p)
    ads[rec["sample_name"]] = ad

list(ads)[:10]


In [ ]:
# Quick sanity check: show spatial for first sample
name0 = next(iter(ads))
ad0 = ads[name0]
name0, ad0


In [ ]:
# Visium stores spot coordinates in ad.obsm['spatial']
print("has spatial coords:", "spatial" in ad0.obsm)
print("obs qc columns:", [c for c in ad0.obs.columns if c.startswith("total_") or c.startswith("n_")][:10])


In [ ]:
# Basic QC plots (per section)
# - spots per section
# - genes per spot distribution

spots_per_section = {k: v.n_obs for k, v in ads.items()}

plt.figure()
plt.bar(range(len(spots_per_section)), list(spots_per_section.values()))
plt.xticks(range(len(spots_per_section)), list(spots_per_section.keys()), rotation=90)
plt.ylabel("Spots")
plt.title("Spots per tissue section (GSE249279)")
plt.tight_layout()
plt.show()


In [ ]:
# Genes per spot: violin per section (scanpy)
# Concatenate just for plotting convenience
ad_all = sc.concat(list(ads.values()), label="section", keys=list(ads.keys()), join="outer", index_unique="-")
sc.pp.calculate_qc_metrics(ad_all, inplace=True)

sc.pl.violin(ad_all, keys=["n_genes_by_counts","total_counts"], groupby="section", rotation=90, show=True)


In [ ]:
# Spatial QC overlay for one section
# total_counts and n_genes_by_counts visualized over tissue
sc.pl.spatial(ad0, color=["total_counts","n_genes_by_counts"], img_key="hires", size=1.3, show=True)


In [ ]:
# Optional: Squidpy graph construction + spatial neighbors (quick check)
sq.gr.spatial_neighbors(ad0, coord_type="grid")
sq.gr.nhood_enrichment(ad0, cluster_key=None)  # no clusters yet; this will be a no-op but confirms squidpy import works
print("squidpy OK")
